# Breast Cancer Wisconsin Data Set

Create a predictive model that classifies benign vs. malignant tumors. 
See https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/data for data understanding.

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

## Load and inspect data set 

In [ ]:
# Fetch the file
my_file = project.get_file("data.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
original_data = pd.read_csv(my_file)

original_data.head()

In [ ]:
original_data = original_data.drop(['Unnamed: 32','id'], axis = 1) # remove unnecessary column with NaN only

In [ ]:
original_data.describe(include='all') # descriptive statistics for all columns

In [ ]:
original_data.isnull().sum() # check for null values

In [ ]:
original_data[original_data.duplicated(keep=False)] # check for duplicate rows

There are no missing values and no duplicates, so you don't have to take actions here. 

## Inspect features

In [ ]:
original_data[['radius_mean', 'diagnosis']].groupby(['diagnosis'], as_index=False).mean().sort_values(by='diagnosis', ascending=False)

Inspect more feature, e.g. texture, perimeter,... 

In [ ]:
# your code

An important step during feature selection is removing features that strongly correlate with each other. You keep only one feature as "representer" of the information and remove redundant features. There are more advanced methods to do this but, for now, just look at the correlation map and decide which features to keep.

In [ ]:
f,ax=plt.subplots(figsize = (18,18))
sns.heatmap(original_data.corr(),annot= True,linewidths=0.5,fmt = ".1f",ax=ax)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title('Correlation Map')
plt.show()

## Select predictors

In [ ]:
data_reduced_features = original_data[['<your feature 1>', '<your feature 2>','...']]

In [ ]:
data_reduced_features.head()

Once again, have a look at the correlation map and remove more features if necessary. 

In [ ]:
f,ax=plt.subplots(figsize = (18,18))
sns.heatmap(data_reduced_features.corr(),annot= True,linewidths=0.5,fmt = ".1f",ax=ax)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title('Correlation Map')
plt.show()

## Prepare for modeling

Set X and y (predictors and target) according to your dataframe:

In [ ]:
target = data_reduced_features['<your target column>']
predictors = # your code

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=123) # 80-20 split into training and test data

Use StandardScaler to scale your predictors (fit on training set and transform training and test set):

In [ ]:
scaler = # your code 
# your code

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Classification models and evaluation metrics

Create a decision tree classifier: 

In [ ]:
# your code
# your code

print('train performance')
print(classification_report(y_train, tree.predict(X_train))) 
print('test performance')
print(classification_report(y_test, tree.predict(X_test)))

How do you evaluate this result (hints: overfitting vs. underfitting, which metric might be important for the use case and why)? 

In [ ]:
conf_mat = confusion_matrix(y_test, tree.predict(X_test))
df_cm = pd.DataFrame(conf_mat, index=['0','1'], columns=['0', '1'],)
fig = plt.figure(figsize=[10,7])
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=14)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)
plt.ylabel('True label')
plt.xlabel('Predicted label')

Create a logistic regression model: 

In [ ]:
# your code
# your code
print('train performance')
print(classification_report(y_train, logreg.predict(X_train)))
print('test performance')
print(classification_report(y_test, logreg.predict(X_test)))

How do you evaluate this result? 

In [ ]:
conf_mat = confusion_matrix(y_test, logreg.predict(X_test))
df_cm = pd.DataFrame(conf_mat, index=['0','1'], columns=['0', '1'],)
fig = plt.figure(figsize=[10,7])
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=14)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)
plt.ylabel('True label')
plt.xlabel('Predicted label')

Feel free to try out more classifiers (don't forget to import required packages!), change classifier parameters, modify train and test split, select other predictors,...  

# Deployment

Deploy the logistic regression and/or decision tree model via the _Watson Machine Learning_ (WML) service on IBM Cloud. Please refer to the documentation for more details about the [watson-machine-learning-client](https://pypi.org/project/watson-machine-learning-client/) or the [REST API](https://watson-ml-api.mybluemix.net/).

In [ ]:
# import the Watson Machine Learning Python client library
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# fill in your credentials
wml_credentials = {
    # your code
}

In [ ]:
# instantiate the client
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# review stored artifacts
wml_client.repository.list()

In [ ]:
# delete artifacts that are no longer needed (note that the number of stored artifacts in the free version is limited to 5)
wml_client.repository.delete("... GUID ...")

In [ ]:
# fill in metadata for your deployment
metadata = {
        wml_client.repository.ModelMetaNames.NAME: 'Breast Cancer Deployment',
        wml_client.repository.ModelMetaNames.DESCRIPTION: 'Test deployment for the breast cancer data set.',
        wml_client.repository.ModelMetaNames.AUTHOR_NAME: 'Your Name'
}

In [ ]:
# choose the model to deploy
model = # your code

In [ ]:
# store the (scikit-learn) model in WML
stored_model = wml_client.repository.store_model(model, meta_props=metadata)

In [ ]:
# get the id of the stored model
published_model_uid = wml_client.repository.get_model_uid(stored_model)

In [ ]:
# create deployment and fetch scoring endpoint
created_deployment = wml_client.deployments.create(published_model_uid, name="Breast Cancer Deployment")
scoring_endpoint = wml_client.deployments.get_scoring_url(created_deployment)

In [ ]:
print(scoring_endpoint)

## Deployment validation

Use the stored deployment to make a prediction.

In [ ]:
# review original data
original_data.head(2)

In [ ]:
# review predictors
predictors.head(2)

In [ ]:
# create scoring payload
scoring_values = predictors.iloc[0:2].to_numpy().tolist()
scoring_payload = {"values": scoring_values}
print(scoring_payload)

In [ ]:
# run prediction
predictions = wml_client.deployments.score(scoring_endpoint, scoring_payload)
print(predictions)

Do the results match your expectation? Were they classified correctly?

In [ ]:
# use the local model to make the same prediction in your notebook and compare the results
model.predict(predictors.iloc[0:2])

Delete the newly created artifacts.